<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/stable/diffusers_inference_ckpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from pathlib import Path
from IPython.display import clear_output
%pip install -q -U --pre triton
%pip install -q accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors xformers
%pip install omegaconf
!pip install diffusers
clear_output(); print('[1;32mDone! ✓')

model_path = "/content/sd_weights_dir"


Done! ✓


In [ ]:
#@title Retrieve ckpt and convert to diffusers
%cd /content
!wget https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_original_stable_diffusion_to_diffusers.py -O convert_original_stable_diffusion_to_diffusers.py

#checkpoint_file = "/content/zkiste3-5454.ckpt"
#!wget https://huggingface.co/steinhaug/zkiste/resolve/main/zkiste3-5454.ckpt -O zkiste3-5454.ckpt

checkpoint_file = "/content/drive/MyDrive/models/zkiste2-3838.ckpt"

!mkdir {model_path}
!python convert_original_stable_diffusion_to_diffusers.py --scheduler_type ddim --checkpoint_path {checkpoint_file} --image_size 512 --prediction_type epsilon --dump_path {model_path} --device cpu
clear_output(); print('[1;32mDone! ✓')

In [3]:
#@title Initialize stable diffusion from folder 
import torch
from torch import autocast
from IPython.display import display

#model_path = "/content/sd_weights_dir"

from diffusers import DiffusionPipeline
pipe = DiffusionPipeline.from_pretrained(model_path, safety_checker=None, torch_dtype=torch.float16).to("cuda")

# DDIMScheduler, DPMSolverMultistepScheduler, EulerDiscreteScheduler, EulerAncestralDiscreteScheduler, DDPMScheduler, PNDMScheduler
from diffusers import DDIMScheduler
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)

pipe.enable_xformers_memory_efficient_attention()
g_cuda = None

g_cuda = torch.Generator(device='cuda')
seed = 1
g_cuda.manual_seed(seed)
saved_file_count = 1;

clear_output(); print('[1;32mDone! ✓')

Done! ✓


In [4]:
YOUR_TOKEN = "zkistebb"
TOKEN_GENDER = "guy"

In [ ]:
#@title AI-Manual - All options mode
prompt = "(full-body full-length full-shot:1.3), __token__ as magician, highly saturated colors, concept art, cottagecore, Gary Larson,cartoonist" #@param {type:"string"}
negative_prompt = "" #@param {type:"string"}
token_name = YOUR_TOKEN + " " + TOKEN_GENDER + ", " + YOUR_TOKEN
num_samples = 1 #@param {type:"number"}
guidance_scale = 8.0 #@param {type:"number"}
num_inference_steps = 50 #@param {type:"number"}
width = "512" #@param ["512", "768", "1280", "1536"] {allow-input: true}
height = "512" #@param ["512", "768", "1280", "1536"] {allow-input: true}
width = int(width)
height = int(height)
custom_seed = 1378417749 #@param {type:"number"}
new_seed = custom_seed
save_images_path = "/content/drive/MyDrive/AI-Images-Manual"

#prompt = prompt.replace("__token__", token_name)

x = token_name.split(",")
for index, value in enumerate(x):
    if index == 0:
        prompt = prompt.replace("__token__", value)
    else:
        prompt = prompt.replace("__token" + str(index) + "__", value)

#raise Exception(1);

if new_seed:
    g_cuda = torch.Generator(device='cuda')
    g_cuda.manual_seed(new_seed)

if len(save_images_path):
    tmp = save_images_path.split("/")
    if len(tmp) == 1:
        save_images_path = "/content/" + save_images_path
    from pathlib import Path
    path = Path(save_images_path)
    if not path.exists():
        print(f"[*] Create save directory...")
        path.mkdir(parents = False, exist_ok = False)
    try:
        if not image_save_count:
            print('Darn we need this one!')
    except NameError:
        image_save_count = 1

print(f"[*] Prompt used: {prompt}")

with autocast("cuda"), torch.inference_mode():
    images = pipe(
        prompt,
        height=height,
        width=width,
        negative_prompt=negative_prompt,
        num_images_per_prompt=num_samples,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=g_cuda
    ).images

for img in images:
    display(img)
    if len(save_images_path):
        precount = f'{image_save_count:04d}'
        image_filename = precount + '_' + prompt.replace(" ", '_')[:240] + '.png'
        img.save(save_images_path + "/" + image_filename)
        image_save_count += 1

In [31]:
#@title Image arrays
auto_prompts = [
    // needs array
]
#  (512,512,256,256,2), (512,1080,128,270,2), (512,768,128,192,2), (512,768,128,192,2), (768,512,192,128,2), (1536,512,384,128,1)
autoresolutions = [
  (512,512,256,256,2)
]

In [ ]:
#@title AI-Automatic - Create some 50+  AI-Images from your brand new trained model
#@markdown I have put together a little "Avatar"-bonus pack for you, press play and let it complete and check your Google Drive folder for the completed AI Images.<br>
#@markdown Time to complete: around XX minutes

import os.path
from os import path
from IPython.display import display
import ast
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

model_trained_steps_for_filename = 'auto'
num_samples = 2
images_savepath = '/content/drive/MyDrive/AI-Images'
#models = "('" + YOUR_TOKEN + "','" + YOUR_TOKEN + " person')"
models = "('" + YOUR_TOKEN + "','" + YOUR_TOKEN + " " + TOKEN_GENDER + "')"

n_override_width = None
n_override_height = None
n_override_seed = None
n_override_guidance_scale = None
n_override_inference_steps = None

width = 512
height = 512

kista_trainednames = ast.literal_eval('[' + models + ']')

if path.exists(images_savepath) == False:
  os.mkdir(images_savepath)



print(f"[*] All AI-Images are saved to your Google Drive in the folder: AI-Images ")
print(f"[*] . ")

for width, height, scaleWidth, scaleHeight, num_samples in autoresolutions:
    print(f"[*] Generating images in {width} x {height}.... ")
    print(f"[*] .. displaying thumbnails of completed images...")
    for prompt_ref,seed,guidance_scale,num_inference_steps,prompt,negative_prompt,filter_x in auto_prompts:
        for modelref,kista_modelname in kista_trainednames:
            if width == 1536:
                if filter_x == 0:
                    continue;
                num_inference_steps = filter_x
            torch.cuda.empty_cache()
            newprompt = prompt.replace("__token__", kista_modelname)
            newprompt = newprompt.replace("__ttoken__", modelref)

            if n_override_seed:
                seed = n_override_seed

            g_cuda = torch.Generator(device='cuda')
            g_cuda.manual_seed(seed)

            if n_override_height:
                height = n_override_height
            if n_override_width:
                width = n_override_width
            if n_override_inference_steps:
                num_inference_steps = n_override_inference_steps
            if n_override_guidance_scale:
                guidance_scale = n_override_guidance_scale
            with autocast("cuda"), torch.inference_mode():
                images = pipe(
                    newprompt,
                    height=height,
                    width=width,
                    negative_prompt=negative_prompt,
                    num_images_per_prompt=num_samples,
                    num_inference_steps=num_inference_steps,
                    guidance_scale=guidance_scale,
                    generator=g_cuda
                ).images
            image_filenames = []
            for img in images:
                prefix_count = '_' + f'{saved_file_count:04d}'
                seed_ref     = '_' + f'{seed:04d}'
                final_id     = 's' + model_trained_steps_for_filename + 'g' + f'{guidance_scale:01f}' + 'i' + f'{num_inference_steps:03d}' + 's' + f'{seed}'
                #outfilename = prompt_ref + '_' + modelref + '_' + final_id.replace(".", '') + prefix_count
                outfilename = prompt_ref + seed_ref + prefix_count
                image_filename = outfilename.replace(" ", '_').replace("00000i", 'i') + '.png'
                img.save(images_savepath + "/" + image_filename)
                image_filenames.append(image_filename)
                saved_file_count += 1
            image_folder = images_savepath + '/'
            grid_row = 1
            grid_col = len(image_filenames)
            grid_scale = 3
            if grid_col > 1:
                fig, axes = plt.subplots(grid_row, grid_col, figsize=(grid_col*grid_scale, grid_row*grid_scale), gridspec_kw={'hspace': 0, 'wspace': 0})
                for j, image_filename in enumerate(image_filenames):
                    currAxes = axes[j]
                    currAxes.set_title(f"{image_filename[0:24]}")
                    image_full_path = os.path.join(image_folder, image_filename)
                    imgdata = mpimg.imread(image_full_path)
                    currAxes.imshow(imgdata, cmap='gray')
                    currAxes.axis('off')
                plt.tight_layout()
                plt.savefig('grid.png', dpi=72)
                plt.show()
            else:
                display(img.resize((scaleWidth, scaleHeight)))


In [ ]:
# RunDiffusion FX Photorealistic
https://civitai.com/api/download/models/88158?type=Model&format=SafeTensor&size=full&fp=fp16

# RunDiffusion FX 2.5D
https://civitai.com/api/download/models/88167?type=Model&format=SafeTensor&size=full&fp=fp16